This program connects to the airline db and runs queries - projection, selection, inner and left join and aggregation.

In [1]:
import sqlite3 as sql
import pandas as pd
conn = sql.connect('airline.db')
cursor = conn.cursor()

Different types of queries:

-  projection
-  selection
-  join (inner join, left join, ~~full join~~)
-  aggregation
-  sub-query
-  set operators

 ### projection
 
 As we know, projection means returning just some of the columns from a dataset.  Here we project just the airline's name, callsign and country.

In [2]:
pd.read_sql_query("select distinct airlinename, callsign, country  from airline \
order by airlinename limit 5",conn)

,AirlineName,Callsign,Country
0,1-2-go,None,Thailand
1,12 North,12N,India
2,135 Airways,GENERAL,United States
3,1Time Airline,NEXTIME,South Africa
4,2 Sqn No 1 Elementary Flying Training School,None,United Kingdom


 ### selection
 
 Selection, in relational algebra, means only returning some of the <em>rows</em> from the dataset.

In [3]:
pd.read_sql_query("select airlinename from airline \
where lower(airlinename) like 'aer%' order by airlinename",conn)

,AirlineName
0,AERFI Group
1,Aer Arann
2,Aer Lingus
3,Aer Turas
4,Aerea Flying Training Organization
...,...
396,Aerovitro
397,Aerovuelox
398,Aeroway
399,Aeroxtra


  ### Joins
    
   - Inner join.  This example joins the route to the airline.  First, we'll select from both, to get the column names.  Then  we'll join the airline's id to the route's airlineid.

In [4]:
pd.read_sql_query("select * from route limit 2", conn)

,index,Code,AirlineID,OriginIATA,OriginID,DestIATA,DestID,Codeshare,StopNums,Equip
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2


In [5]:
pd.read_sql_query("select * from airline limit 2", conn)

,index,id,AirlineName,Alias,IATA,ICAO,Callsign,Country,Active
0,0,-1,Unknown,\N,-,None,\N,\N,Y
1,1,1,Private flight,\N,-,None,None,None,Y


Now, the inner join

In [6]:
pd.read_sql_query("select AirlineName, OriginIATA, DestIATA from route \
join airline on (airline.id = route.airlineid) limit 20", conn)

,AirlineName,OriginIATA,DestIATA
0,Aerocondor,AER,KZN
1,Aerocondor,ASF,KZN
2,Aerocondor,ASF,MRV
3,Aerocondor,CEK,KZN
4,Aerocondor,CEK,OVB
5,Aerocondor,DME,KZN
6,Aerocondor,DME,NBC
7,Aerocondor,DME,TGK
8,Aerocondor,DME,UUA
9,Aerocondor,EGO,KGD


 Left join - let's see if there are any airlines with no routes.

In [7]:
pd.read_sql_query("select  id, AirlineName, OriginIATA, DestIATA from airline \
left join route on (airline.id = route.airlineid)  \
where  DestIATA is null \
 limit 20", conn)

,id,AirlineName,OriginIATA,DestIATA
0,-1,Unknown,None,None
1,1,Private flight,None,None
2,2,135 Airways,None,None
3,3,1Time Airline,None,None
4,4,2 Sqn No 1 Elementary Flying Training School,None,None
5,5,213 Flight Unit,None,None
6,6,223 Flight Unit State Airline,None,None
7,7,224th Flight Unit,None,None
8,8,247 Jet Ltd,None,None
9,9,3D Aviation,None,None


 ### Left join
    - We'll use the left join to see which airlines have no routes

 What about routes with no airlines?  SQLite doesn't support right joins or full joins, so we can use a left join, swapping the order of the tables.

In [ ]:
pd.read_sql_query("select  airlineid, AirlineName, OriginIATA, DestIATA from route \
left join airline on (airline.id = route.airlineid)  \
where  airlinename is null  \
 limit 20", conn)

So we're getting  '\\N' in some columns and None in another.  Let's check to see what the data for one of the routes returned is.  We'll take the one with origin IATA AKV and destination IATA YIK

In [ ]:
pd.read_sql_query("select  airlineid, AirlineName, OriginIATA, DestIATA from route \
left join airline on (airline.id = route.airlineid)  \
where  originIATA='AKV' and DestIATA = 'YIK'  \
 limit 20", conn)

That's telling us that many routes have no airline operating them.  I'll check the number, by leaving off the 'limit'

In [ ]:
pd.read_sql_query("select * from route where AirlineId = '\\N' \
", conn)

 ### Aggregation

So, let's see how many airlines operate fewer than 3 routes

In [ ]:
pd.read_sql_query("select airlinename, count(originid) NoRoutes from route \
left join airline on (route.airlineid = airline.id) \
group by airlinename \
having count(originid) < 3 \
order by NoRoutes", conn)

Eurowings seems to be operating a route - let's see what it is.

In [ ]:
pd.read_sql_query("select * from airline join route on (id=airlineid) \
where airlinename = 'Eurowings'", conn)

 ### Sub-query
 
Let's look at all the routes operated by our few-route airlines

In [ ]:
pd.read_sql_query("select * from airline join route on (id=airlineid) \
where airlinename in (select airlinename \
 from route \
left join airline on (route.airlineid = airline.id) \
group by airlinename \
having count(originIATA) < 3) \
order by airlinename", conn)

 ### Views


    We often use views when using set operators.  We can check to see what's in our database and we can create a view.
    
    Let's make a view of Ryanair routes.  First we'll display them, then we'll get the airport info and make a view.

In [ ]:
pd.read_sql_query("select * from route join airline on (route.airlineid = airline.id) \
where lower(airlinename) like 'ryanair'",conn)

Make it easier to read, by adding the airport information from origin and destination.

In [ ]:
pd.read_sql_query("select \
oa.name FromPort, oa.city FromCity, oa.country FromCountry, \
da.name ToPort, da.city ToCity, da.country ToCountry \
from route \
join airline on (route.airlineid = airline.id) \
join airport oa on (route.originid = oa.id) \
join airport da on (route.destid=da.id) \
where lower(airlinename) like 'ryanair'",conn)

Now let's make a view of it.

In [ ]:
cursor.execute("Drop view if exists raroute")
conn.execute(""" create view RAroute as select \
oa.name FromPort, oa.city FromCity, oa.country FromCountry, \
da.name ToPort, da.city ToCity, da.country ToCountry \
from route \
join airline on (route.airlineid = airline.id) \
join airport oa on (route.originid = oa.id) \
join airport da on (route.destid=da.id) \
where lower(airlinename) like 'ryanair'""")

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type in ('table','view');")
print(cursor.fetchall())

In [ ]:
pd.read_sql_query("select count(*) from raroute", conn)

In [ ]:
pd.read_sql_query("select * from raroute limit 10", conn)

In [ ]:
pd.read_sql_query("select * from raroute where ToCountry = 'Spain'", conn)

In [ ]:
pd.read_sql_query("select FromCountry, ToCountry from raroute", conn)

In [ ]:
pd.read_sql_query("select FromCountry, ToCountry, count(*) NoRoutes \
from raroute group by FromCountry, ToCountry order by NoRoutes\
", conn)

In [ ]:
pd.read_sql_query("select * from  raroute where FromCountry = 'Cyprus' ", conn)

In [ ]:
pd.read_sql_query("select * from  raroute where FromCountry = 'Latvia' ", conn)

To prepare for our set operations, let's  get some sets.  e.g. 
 - Pick out the Ryanair routes.

In [ ]:
pd.read_sql_query("select ToCountry, count(*) NoRoutes from  raroute \
where FromCountry = 'Ireland' \
group by ToCountry", conn)

In [ ]:
conn.close()